In [1]:
import yaml

with open('configs/paths.yaml', 'r') as stream:
    PATHS = yaml.safe_load(stream)

MAIN_ACTORS = ['Chandler', 'Joey', 'Monica', 'Phoebe', 'Rachel', 'Ross']

FACE_EMBEDDINGS_DIR = './main-actors/'

ANGLE_DIST_THRESHOLD = 1.15

VID_WIDTH = 224
VID_HEIGHT = 224

In [2]:
import numpy as np
import os
from glob import glob
import csv
from tqdm.notebook import tqdm

with open(PATHS['ANNOTATIONS']['all'], 'r') as stream:
    datasets = yaml.safe_load(stream)

face_embeddings = {os.path.basename(filename).split('.npy')[0]: np.load(filename).reshape(1, 512) 
                   for filename in glob(os.path.join(FACE_EMBEDDINGS_DIR, '*.npy'))}

MAIN_ACTORS = sorted([foo.split('.npy')[0] for foo in os.listdir(FACE_EMBEDDINGS_DIR)])

In [3]:
from cltl_face_all.face_alignment import FaceDetection
import av
from av import InvalidDataError
from cltl_face_all.arcface import calc_angle_distance

fd = FaceDetection(device='cpu', face_detector='sfd')

for DATASET in tqdm(['train', 'dev', 'test']):
    os.makedirs(os.path.join(PATHS['FACE_VIDEOS'][DATASET]), exist_ok=True)

    for diautt, anno in tqdm(datasets[DATASET].items()):
        try:
            speaker = datasets[DATASET][diautt]['Speaker']
        except KeyError as e:
            print(f"{diautt} not in {DATASET}, {e}")

        if speaker not in MAIN_ACTORS:
            continue

        vidpath = os.path.join(PATHS['ORIGINAL_VIDEOS'][DATASET], diautt) + '.mp4'
        try:
            container = av.open(vidpath)
            fps = round(float(container.streams.video[0].average_rate))
        except InvalidDataError as e:
            print(e)
            continue

        try:
            features = np.load(os.path.join(PATHS['VISUAL_FEATURES'][DATASET], 
                                f'{diautt}.npy'), allow_pickle=True).item()
        except Exception as e:
            print(f"Couldn't load the features {e}")

        faces = []
        for idx, frame in enumerate(container.decode(video=0)):
            feature = features[idx]

            if len(feature) == 0:
                continue

            numpy_RGB = np.array(frame.to_image())
            batch = numpy_RGB[np.newaxis, ...]

            for feat in feature:
                bbox = feat['bbox']
                landmark = feat['landmark']
                age = feat['age']
                gender = feat['gender']
                embedding = feat['embedding']
                embedding = embedding.reshape(1, 512)

                dist = calc_angle_distance(embedding, face_embeddings[speaker])
                dist = float(dist[0, 0])

                if dist >= ANGLE_DIST_THRESHOLD:
                    continue

                face = fd.crop_and_align(batch, [bbox[np.newaxis, ...]], [landmark[np.newaxis, ...]],
                                         crop_size=VID_WIDTH)
                face = np.squeeze(face)
                faces.append(face)

        if len(faces) == 0:
            continue
    
        save_full_path = os.path.join(PATHS['FACE_VIDEOS'][DATASET], diautt) + '.mp4'
        container = av.open(save_full_path, mode='w')

        stream = container.add_stream('mpeg4', rate=fps)
        stream.width = VID_WIDTH
        stream.height = VID_HEIGHT

        for face in faces:
            frame = av.VideoFrame.from_ndarray(face, format='rgb24')
            for packet in stream.encode(frame):
                container.mux(packet)
        # Flush stream
        for packet in stream.encode():
            container.mux(packet)

        # Close the file
        container.close()

too many threads/slices (15), reducing to 14
too many threads/slices (15), reducing to 14
too many threads/slices (15), reducing to 14
too many threads/slices (15), reducing to 14
too many threads/slices (15), reducing to 14
too many threads/slices (15), reducing to 14
too many threads/slices (15), reducing to 14
too many threads/slices (15), reducing to 14
too many threads/slices (15), reducing to 14
too many threads/slices (15), reducing to 14
too many threads/slices (15), reducing to 14
too many threads/slices (15), reducing to 14
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/tk/.virtualenvs/dev-python-3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-79d20568caa0>", line 35, in <module>
    for idx, frame in enumerate(container.decode(video=0)):
KeyboardIn

TypeError: object of type 'NoneType' has no len()